In [55]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [56]:
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.model_selection import KFold
from datetime import datetime
from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import math

#basic tools 
import os
import numpy as np
import pandas as pd
import warnings

#tuning hyperparameters
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

#graph, plots
import matplotlib.pyplot as plt
import seaborn as sns

#building models
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys

#metrics 
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
import shap
warnings.simplefilter(action='ignore', category=FutureWarning)

In [57]:
train = pd.read_csv("../input/dataverse/train.csv")
test = pd.read_csv('../input/dataverse/test.csv')
submission = pd.read_csv("../input/dataverse/sample_submission.csv")

In [58]:
train.shape, test.shape, submission.shape

((58592, 44), (39063, 43), (39063, 2))

In [59]:
train.head(2)

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0


In [60]:
train['is_claim'].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

In [61]:
test.head(2)

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,ID58593,0.341732,0.00,0.586538,C3,4076,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
1,ID58594,0.307241,0.13,0.442308,C8,8794,1,B2,M6,Petrol,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2


In [62]:
submission.head(2)

,policy_id,is_claim
0,ID58593,1
1,ID58594,1


In [63]:
train_x = train.iloc[:,:-1]
test_x = test.iloc[:,:]
train_y = train['is_claim']
total_data = pd.concat([train_x,test_x],axis=0)

In [64]:
total_data.shape

(97655, 43)

In [65]:
total_data.head(3)

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0


In [66]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97655 entries, 0 to 39062
Data columns (total 43 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         97655 non-null  object 
 1   policy_tenure                     97655 non-null  float64
 2   age_of_car                        97655 non-null  float64
 3   age_of_policyholder               97655 non-null  float64
 4   area_cluster                      97655 non-null  object 
 5   population_density                97655 non-null  int64  
 6   make                              97655 non-null  int64  
 7   segment                           97655 non-null  object 
 8   model                             97655 non-null  object 
 9   fuel_type                         97655 non-null  object 
 10  max_torque                        97655 non-null  object 
 11  max_power                         97655 non-null  object 
 12  engi

In [67]:
total_data['m_s_m_f_t'] = total_data['make'].astype(str)+'_'+total_data['segment'].astype(str) + '_'+total_data['model'].astype(str)+'_'+total_data['fuel_type'].astype(str)+'_'+total_data['transmission_type'].astype(str)
total_data['a_p'] = total_data['area_cluster'].astype(str)+'_'+total_data['population_density'].astype(str)


In [68]:
total_data.head(2)

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,m_s_m_f_t,a_p
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,Yes,No,No,No,Yes,0,1_A_M1_CNG_Manual,C1_4990
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,Yes,No,No,No,Yes,0,1_A_M1_CNG_Manual,C2_27003


In [69]:
encoding_columns = ['make','area_cluster','segment','model','fuel_type','max_torque','max_power',
                   'engine_type','is_esc','is_adjustable_steering','is_tpms','is_parking_sensors',
                   'is_parking_camera','rear_brakes_type','transmission_type','steering_type','is_front_fog_lights',
                   'is_rear_window_wiper','is_rear_window_washer','is_rear_window_defogger','is_brake_assist','is_power_door_locks',
'is_central_locking','is_power_steering','is_driver_seat_height_adjustable','is_day_night_rear_view_mirror',
'is_ecw','is_speed_alert','m_s_m_f_t','a_p']

for each_col in tqdm(encoding_columns):
    label_encoder = preprocessing.LabelEncoder()
    total_data[each_col]= label_encoder.fit_transform(total_data[each_col])

100%|██████████| 30/30 [00:00<00:00, 43.21it/s]


In [70]:
total_data.head(2)

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,m_s_m_f_t,a_p
0,ID00001,0.515874,0.05,0.644231,0,4990,0,0,0,0,...,0,0,1,0,0,0,1,0,0,10
1,ID00002,0.672619,0.02,0.375000,11,27003,0,0,0,0,...,0,0,1,0,0,0,1,0,0,14


In [71]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97655 entries, 0 to 39062
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         97655 non-null  object 
 1   policy_tenure                     97655 non-null  float64
 2   age_of_car                        97655 non-null  float64
 3   age_of_policyholder               97655 non-null  float64
 4   area_cluster                      97655 non-null  int64  
 5   population_density                97655 non-null  int64  
 6   make                              97655 non-null  int64  
 7   segment                           97655 non-null  int64  
 8   model                             97655 non-null  int64  
 9   fuel_type                         97655 non-null  int64  
 10  max_torque                        97655 non-null  int64  
 11  max_power                         97655 non-null  int64  
 12  engi

In [72]:
best_cols = list(set(['policy_tenure',
 'age_of_car',
 'age_of_policyholder',
 'area_cluster',
 'population_density',
 'make',
 'segment',
 'model',
 'fuel_type',
 'max_torque',
 'max_power',
 'engine_type',
 'airbags',
 'is_parking_camera',
 'displacement',
 'cylinder',
 'steering_type',
 'turning_radius',
 'length',
 'width',
 'height',
 'gross_weight',
 'is_front_fog_lights',
 'is_brake_assist',
 'ncap_rating',
 'm_s_m_f_t',
 'a_p']))

In [73]:
len(best_cols)

27

In [74]:
total_data = total_data.loc[:,best_cols]
total_data.shape

(97655, 27)

In [75]:
total_data.head(2)

,steering_type,age_of_policyholder,height,max_power,population_density,airbags,is_front_fog_lights,is_brake_assist,policy_tenure,displacement,...,area_cluster,ncap_rating,length,fuel_type,make,engine_type,gross_weight,model,cylinder,width
0,2,0.644231,1475,2,4990,2,0,0,0.515874,796,...,0,0,3445,0,0,6,1185,0,3,1515
1,2,0.375000,1475,2,27003,2,0,0,0.672619,796,...,11,0,3445,0,0,6,1185,0,3,1515


In [76]:
train_x = total_data.iloc[:58592,:]
test_x = total_data.iloc[58592:,:]

In [77]:
train_x.shape, train_y.shape, test_x.shape

((58592, 27), (58592,), (39063, 27))

In [78]:
test_x.shape

(39063, 27)

In [79]:
train_x.shape, train_y.shape

((58592, 27), (58592,))

In [80]:
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=3, random_seed=6,n_estimators=10000, output_process=False):
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 1.0),
                                            'num_leaves': (24, 80),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 30),
                                            'max_bin':(20,90),
                                            'min_data_in_leaf': (20, 80),
                                            'min_sum_hessian_in_leaf':(0,100),
                                           'subsample': (0.01, 1.0)}, random_state=200)

    
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    return lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']



In [81]:
%%time
opt_params = bayes_parameter_opt_lgb(train_x, train_y, init_round=5, opt_round=10, n_folds=3, random_seed=6,n_estimators=10000)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9895264513703341, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9895264513703341
[LightGBM] [Warning] bagging_fraction is set=0.9895264513703341, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9895264513703341
[LightGBM] [Warning] bagging_fraction is set=0.9895264513703341, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9895264513703341
[LightGBM] [Info] Number of positive: 2499, number of negative: 36562
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memo

'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Fatal] Cannot change max_bin after constructed Dataset handle.


[LightGBM] [Warning] bagging_fraction is set=0.9963605418316657, subsample=0.2580000042232922 will be ignored. Current value: bagging_fraction=0.9963605418316657
[LightGBM] [Warning] bagging_fraction is set=0.9963605418316657, subsample=0.2580000042232922 will be ignored. Current value: bagging_fraction=0.9963605418316657
[LightGBM] [Info] Number of positive: 2499, number of negative: 36562
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018559 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 224
[LightGBM] [Info] Number of data points in the train set: 39061, number of used features: 27
[LightGBM] [Warning] bagging_fraction is set=0.9963605418316657, subsample=0.2580000042232922 will be ignored. Current value: bagging_fraction=0.9963605418316657
[LightGBM] [Warning] bagging_fraction is set=0.9963605418316657, subsample=0.2580000042232922 will be ignored. Current value: bagging_fraction=0.99636054

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Fatal] Cannot change max_bin after constructed Dataset handle.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019485 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 292
[LightGBM] [Info] Number of data points in the train set: 39061, number of used features: 27
[LightGBM] [Warning] bagging_fraction is set=0.9902482318856642, subsample=0.1653821321337434 will be ignored. Current value: bagging_fraction=0.9902482318856642
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] bagging_fraction is set=0.9902482318856642, subsample=0.1653821321337434 will be ignored. Current value: bagging_fraction=0.9902482318856642
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] bagging_fraction is set=0.9902482318856642, subsample=0.1653821321337434 will be ignored. Current value:

'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


|  15       |  0.6444   |  0.9902   |  0.8095   |  0.4531   |  89.57    |  24.81    |  54.71    |  5.316    |  31.13    |  0.1654   |
CPU times: user 1min 11s, sys: 1.51 s, total: 1min 13s
Wall time: 1min 12s


In [82]:
opt_params[1]["num_leaves"] = int(round(opt_params[1]["num_leaves"]))
opt_params[1]['max_depth'] = int(round(opt_params[1]['max_depth']))
opt_params[1]['min_data_in_leaf'] = int(round(opt_params[1]['min_data_in_leaf']))
opt_params[1]['max_bin'] = int(round(opt_params[1]['max_bin']))
opt_params[1]['objective']='binary'
opt_params[1]['metric']='auc'
opt_params[1]['is_unbalance']=True
opt_params[1]['boost_from_average']=False
opt_params=opt_params[1]
opt_params

{'bagging_fraction': 0.8333129803667263,
 'feature_fraction': 0.7175451796385642,
 'learning_rate': 0.06082589049721443,
 'max_bin': 87,
 'max_depth': 25,
 'min_data_in_leaf': 63,
 'min_sum_hessian_in_leaf': 32.902989847268,
 'num_leaves': 30,
 'subsample': 0.2623276224265793,
 'objective': 'binary',
 'metric': 'auc',
 'is_unbalance': True,
 'boost_from_average': False}

In [83]:
train_x.shape, train_y.shape, test_x.shape

((58592, 27), (58592,), (39063, 27))

In [84]:
%%time 

target=train_y
features= [c for c in train_x.columns]


folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=31416)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x.values, train_y.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_x.iloc[trn_idx][features], label=train_y[trn_idx])
    val_data = lgb.Dataset(train_x.iloc[val_idx][features], label=train_y[val_idx])

    num_round = 15000
    clf = lgb.train(opt_params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 500)
    oof[val_idx] = clf.predict(train_x.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_x[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

Fold 0
[LightGBM] [Warning] bagging_fraction is set=0.8333129803667263, subsample=0.2623276224265793 will be ignored. Current value: bagging_fraction=0.8333129803667263


'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] bagging_fraction is set=0.8333129803667263, subsample=0.2623276224265793 will be ignored. Current value: bagging_fraction=0.8333129803667263
[LightGBM] [Info] Number of positive: 3373, number of negative: 49359
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 52732, number of used features: 27
[LightGBM] [Warning] bagging_fraction is set=0.8333129803667263, subsample=0.2623276224265793 will be ignored. Current value: bagging_fraction=0.8333129803667263
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[94]	training's auc: 0.733603	valid_1's auc: 0.656463
Fold 1
[LightGBM] [Warning] bagging_fraction is set=0.8333129803667263, subsample=0.2623276224265793 will be ignored. Current value: bagging_fraction=0.833312980366

In [85]:
type(predictions)

numpy.ndarray

In [86]:
predictions.shape

(39063,)

In [87]:
predictions[0:10]

array([0.63159361, 0.49732814, 0.43270252, 0.42568395, 0.52554274,
       0.43237071, 0.53803797, 0.60576357, 0.30565015, 0.53952536])

In [94]:
sub = pd.read_csv("../input/dataverse/sample_submission.csv")
sub.head(2)

,policy_id,is_claim
0,ID58593,1
1,ID58594,1


In [95]:
sub['is_claim'] = predictions
sub['is_claim'] = sub['is_claim'].apply(lambda x : 1 if x > 0.6 else 0)
sub['is_claim'].value_counts()

0    36507
1     2556
Name: is_claim, dtype: int64

In [96]:
sub.to_csv("lgb_51.csv",index=False)

In [97]:
# cols = (feature_importance_df[["Feature", "importance"]]
#         .groupby("Feature")
#         .mean()
#         .sort_values(by="importance", ascending=False)[:30].index)
# best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

# plt.figure(figsize=(20,28))
# sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
# plt.title('Features importance (averaged/folds)')
# plt.tight_layout()
